In [1]:
import time
from math import floor
from rubik_cubes import Rubik, rubik_corner, rubik_edge
from rubik_state import rubik_state
from rubik_next import get_nexts_1, get_nexts_2
from rubik_file import RubikFile
from rubik_ida import IDA
from itertools import permutations
import copy

In [2]:
rubikfile = RubikFile("test.h5")
rubik = Rubik()

In [3]:
# solved = rubik_state(rubik, 0, rubik.heuristic_h1)

In [38]:
def brute_force_co(save, solved):
    current = solved
    save[current.corners_state] = current.g
    queue = [current]
    
    while len(queue):
        current = queue.pop(0)
        for elem in get_nexts_1(current, null_h):
            if elem.corners_state not in save.keys():
                save[elem.corners_state] = elem.g
                queue.append(elem)

def brute_force_eo(save, solved):
    current = solved
    save[current.edges_state] = current.g
    queue = [current]
    
    while len(queue):
        current = queue.pop(0)
        for elem in get_nexts_1(current, null_h):
            if elem.edges_state not in save.keys():
                save[elem.edges_state] = elem.g
                queue.append(elem)

def brute_force_slice(save, solved):
    current = solved
    perm = permutations([5, 6, 7, 8])
    queue = [current]
    for elem in perm:
        new_dep = copy.deepcopy(solved)
        for i in range(5, 9):
            new_dep.corners[i].position = elem[i - 5]
        queue.append(new_dep)
        new_dep.slice_state = elem
        save[new_dep.slice_state] = 0
    save[current.slice_state] = current.g
    while len(queue):
        current = queue.pop(0)
        for elem in get_nexts_1(current, null_h):
            if elem.slice_state not in save.keys():
                save[elem.slice_state] = elem.g
                queue.append(elem)

def null_h(obj):
    return 0

In [36]:
solved = rubik_state(rubik, 0, null_h)
save_slice = {}
brute_force_slice(save_slice, solved)

In [39]:
solved = rubik_state(rubik, 0, null_h)
save_co = {}
brute_force_co(save_co, solved)

In [40]:
solved = rubik_state(rubik, 0, null_h)
save_eo = {}
brute_force_eo(save_eo, solved)

In [41]:
print(sum([i for i in save_co.values()]) / len(save_co))
print(len(save_co))
print(sum([i for i in save_eo.values()]) / len(save_eo))
print(len(save_eo))
print(sum([i for i in save_slice.values()]) / len(save_slice))
print(len(save_slice))

4.459076360310928
2187
4.60888671875
2048
5.491237882177479
10728


In [43]:
rubikfile = RubikFile("pruning_phase_1.h5")
rubikfile.write("corners", save_co)

Writing...
Write done


In [44]:
rubikfile.write("edges", save_eo)

Writing...
Write done


In [45]:
rubikfile.write("slice", save_slice)

Writing...
Write done


In [2]:
# phase two generator, to be reworked
def compress_state(state):
    compressed = 0
    # first compress corners place
    for i in range(1, 9):
        compressed <<= 3
        compressed |= (state.corners[i].final_position - 1)
    # compress corners orientation
    for i in range(1, 9):
        compressed <<= 2
        compressed |= state.corners[i].orientation
    # compress edges place
    for i in range(1, 13):
        compressed <<= 4
        compressed |= state.edges[i].final_position
    # corners edges orientation
    for i in range(1, 13):
        compressed <<= 1
        compressed |= state.edges[i].orientation
    return str(compressed)

def gen_pruning(dico, curr_node, depth, h):
    if depth < 9:
        next_depth = []
        for elem in get_nexts_2(curr_node, h):
            cp = compress_state(elem)
            if cp not in dico.keys():
                dico[cp] = depth + 1
                next_depth.append(elem)
            elif dico[cp] > depth + 1:
                dico[cp] = depth + 1
        for elem in next_depth:
            gen_pruning(dico, elem, depth + 1, h)

dico = {}
rubik = Rubik()
dep = rubik_state(rubik, 0, rubik.heuristic_h2)
dico[compress_state(dep)] = 0

start = time.time()
gen_pruning(dico, dep, 0, rubik.heuristic_h2)
print(time.time() - start)
print(len(dico))

In [26]:
rubikfile.write("phase2", dico)

pruning_phase2 = rubikfile.read("phase2")
len(pruning_phase2)

546292

In [ ]:
# def heuristic_corners_orientation(obj):
#     tot = 0
#     for elem in obj.corners.keys():
#         if obj.corners[elem].orientation != 0:
#             tot += 1
#     return tot / 4
# def transform_number_in_base_x_tab(nb, base, len_tab):
#     tab = []
#     for i in range(0, len_tab):
#         tab.insert(0, nb % base)
#         nb = floor(nb / base)
#     return tab

# def check_parity(tab, base):
#     if sum(tab) % 3 == 0:
#         return True
#     return False

# def modify_orientation(state, tab):
#     for i in range(len(tab)):
#         state.corners[i + 1].orientation = tab[i]

In [16]:
solved = rubik_state(solved, 14, null_h)
tuple([elem[1].final_position for elem in solved.edges.items() if elem[0] in [5, 6, 7, 8]])

(6, 5, 7, 8)

In [17]:
tuple([solved.edges[i].final_position for i in range(5, 9)])

(6, 5, 7, 8)

In [20]:
start = time.time()
for i in range(0, 1000000):
    tmp = tuple([elem[1].final_position for elem in solved.edges.items() if elem[0] in [5, 6, 7, 8]])
print(time.time() - start)

2.0992629528045654


In [19]:
perm = permutations([5, 6, 7, 8])
for elem in perm:
    new_dep = copy.deepcopy(solved)
    for i in range(5, 9):
        new_dep.edges[i].final_position = elem[i - 5]
    print(elem)
    print(new_dep)

(5, 6, 7, 8)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 5 6 7 8 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1 2 3 4 5 6 7 8 
(5, 6, 8, 7)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 5 6 8 7 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1 2 3 4 5 6 7 8 
(5, 7, 6, 8)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 5 7 6 8 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1 2 3 4 5 6 7 8 
(5, 7, 8, 6)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 5 7 8 6 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1 2 3 4 5 6 7 8 
(5, 8, 6, 7)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 5 8 6 7 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1 2 3 4 5 6 7 8 
(5, 8, 7, 6)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 5 8 7 6 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1 2 3 4 5 6 7 8 
(6, 5, 7, 8)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 6 5 7 8 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1 2 3 4 5 6 7 8 
(6, 5, 8, 7)
inst: None
eo: 0 0 0 0 0 0 0 0 0 0 0 0 
ep: 1 2 3 4 6 5 8 7 9 10 11 12 
co: 0 0 0 0 0 0 0 0 
cp: 1

(5, 6, 7, 8)
(5, 6, 8, 7)
(5, 7, 6, 8)
(5, 7, 8, 6)
(5, 8, 6, 7)
(5, 8, 7, 6)
(6, 5, 7, 8)
(6, 5, 8, 7)
(6, 7, 5, 8)
(6, 7, 8, 5)
(6, 8, 5, 7)
(6, 8, 7, 5)
(7, 5, 6, 8)
(7, 5, 8, 6)
(7, 6, 5, 8)
(7, 6, 8, 5)
(7, 8, 5, 6)
(7, 8, 6, 5)
(8, 5, 6, 7)
(8, 5, 7, 6)
(8, 6, 5, 7)
(8, 6, 7, 5)
(8, 7, 5, 6)
(8, 7, 6, 5)
